In [8]:
import time
import matplotlib.pyplot as plt
import numpy as np
from skimage import io, morphology, exposure, color
import random
import sys
from math import ceil,floor
import logging
from logging.config import dictConfig
from google.colab import drive
import copy
import cv2
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [27]:
def textureGrowth(imageFile, windowSize, Path):

    img = imageFile
    #reducing image to 0-1 range
    img = img/255.0     
    print(np.shape(img))
           
    row, col = np.shape(img)
    Error_Threshold = 0.1
    Maximum_Error_Threshold = 0.3
    sigma = windowSize/6.4
    seed = 3
    halfWindow = windowSize// 2
    #total filled pixels
    row,col = img.shape
    totalPixels = row*col
    

    synthesizedImage = copy.copy(img)
    filledMap = np.ceil(img)
    filledPixels = np.sum(filledMap)
    bestNeigh = sliding_window_object_removal(img,filledMap,halfWindow)

    #adding padding to handle edge cases
    synthImagePad = np.lib.pad(synthesizedImage, halfWindow, mode='constant', constant_values=0)
    filledMapPad = np.lib.pad(filledMap, halfWindow, mode='constant', constant_values=0)

    gaussMask = GaussMask(windowSize,sigma)

    # Run a while loop till all our pixels are filled
    while filledPixels < totalPixels:
        progress = False

        pixelList = np.nonzero(morphology.binary_dilation(filledMap) - filledMap)
        neighbors = []
        neighbors.append([np.sum(filledMap[pixelList[0][i] - halfWindow : pixelList[0][i] + halfWindow + 1, pixelList[1][i] - halfWindow : pixelList[1][i] + halfWindow + 1]) for i in range(len(pixelList[0]))])
        l1 = np.argsort(-np.array(neighbors, dtype=int)) #unfilled neighbours in descending order

        for i in l1[0]:

            template = synthImagePad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            validMask = filledMapPad[pixelList[0][i] - halfWindow + halfWindow:pixelList[0][i] + halfWindow + halfWindow + 1, pixelList[1][i] - halfWindow + halfWindow:pixelList[1][i] + halfWindow + halfWindow + 1]
            bestMatches = findMatches(template,bestNeigh,validMask,gaussMask,windowSize, halfWindow, Error_Threshold)
            # Random picking of best matches
            bestMatch = random.randint(0, len(bestMatches)-1)
            if bestMatches[bestMatch][0]<=Maximum_Error_Threshold:
                synthImagePad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = bestMatches[bestMatch][1]
                synthesizedImage[pixelList[0][i]][pixelList[1][i]] = bestMatches[bestMatch][1]
                filledMapPad[halfWindow+pixelList[0][i]][halfWindow+pixelList[1][i]] = 1
                filledMap[pixelList[0][i]][pixelList[1][i]] = 1
                filledPixels+=1
                progress = True
        if not progress:
            #Increainf threshold if no progress.
            Maximum_Error_Threshold *= 1.1
        i = (filledPixels/totalPixels)*100
        sys.stdout.write("\r%d%%" % i)
        sys.stdout.flush()
    io.imsave(Path.split('.')[0]+"-"+str(windowSize)+"-question3.bmp", synthesizedImage*255.0)
    plt.show()
    return


In [3]:
def GaussMask(windowSize, sigma):
    x, y = np.mgrid[-windowSize//2 + 1:windowSize//2 + 1, -windowSize//2 + 1:windowSize//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g/g.sum()


In [6]:

def sliding_window_object_removal(SampleImage, is_filled,n):
    src_window_matrix = []
    c = 0
    for i in range(n, SampleImage.shape[0] - n - 1):
        for j in range(n, SampleImage.shape[1] - n - 1):
            if 0 in is_filled[i - n:i + n + 1, j - n: j + n + 1]:
                c = c + 1
            else:
                src_window_matrix.append(np.reshape(
                    SampleImage[i - n:i + n + 1, j - n: j + n + 1],
                    (2 * n + 1) ** 2))
    
    return np.double(src_window_matrix)


In [4]:

def findMatches(template,bestNeigh, valid_mask, gauss_mask, windowSize, halfWindow, Error_Threshold):
    template = np.reshape(template, windowSize*windowSize)
    gauss_mask = np.reshape(gauss_mask, windowSize*windowSize)
    valid_mask = np.reshape(valid_mask, windowSize*windowSize)
    total_weight = np.sum(np.multiply(gauss_mask, valid_mask))
    distance = (bestNeigh-template)**2
    ssd = np.sum((distance*gauss_mask*valid_mask) / total_weight, axis=1)
    min_error = min(ssd)
    j = int(((2 * halfWindow + 1) ** 2) / 2)
    return [[err, bestNeigh[i][j]] for i, err in enumerate(ssd) if err <= min_error*(1+Error_Threshold)]



In [19]:
def preprocess(file_input,mask_input):
  img_grayscale = color.rgb2gray(io.imread(file_input))
  img_mask = color.rgb2gray(cv2.imread(mask_input))
  print(np.shape(img_grayscale))

  x = np.where(img_mask>0.9)
  for i in range(len(x[0])):
    img_grayscale[x[0][i]][x[1][i]]=0
    img_mask[x[0][i]][x[1][i]]=1

    img_mask = np.floor(img_mask)

  return img_grayscale,img_mask





In [31]:
if __name__ == '__main__':

    logging.basicConfig(filename="runTime.log", level=logging.INFO)

    filePath = "/content/gdrive/My Drive/Colab Notebooks/"

    for windowSize in [9]:
        start = time.time()
        file_input="/content/gdrive/My Drive/Colab Notebooks/test_im3.bmp"
        mask_input="/content/gdrive/My Drive/Colab Notebooks/mask1.bmp"
        ip,m=preprocess(file_input,mask_input)
        textureGrowth(ip, windowSize,"/content/gdrive/My Drive/Colab Notebooks/test_im3.bmp")
        end = time.time()
            #Calculate and log the running times
        logging.info("\t"+file_input+"-"+str(windowSize)+"\t:-  "+str(end-start)+" secs")


(667, 1000)
(667, 1000)
96%

KeyboardInterrupt: ignored